In [0]:
import pandas as pd
import math
import csv
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pylab as plt
import csv
import os

from scipy.stats import gaussian_kde
from scipy.stats import powerlaw

In [0]:
df = pd.read_csv('allgraph.csv', sep='\t', header=None, encoding='utf-8', engine='python')

In [0]:
graph = df.rename(columns={0:'KEYWORD',1:'POSITION',2:'SERP'})

In [0]:
weight = {1:1,2:1,3:1,4:0.85,5:0.6,6:0.5,7:0.5,8:0.3,9:0.3,10:0.2,11:0.15,12:0.15,13:0.10,14:0.10,15:0.08,16:0.05,17:0.03,18:0.02,19:0.01,20:0.009}
graph['POSITION'] = graph['POSITION'].replace(weight, regex=True)

In [0]:
set_k = set(graph['KEYWORD'])
set_s = set(graph['SERP'])

In [0]:
authority = {}
hub = {}

for elem in set_k:
    hub[elem] = 0
    
for elem in set_s:
    authority[elem] = 0

In [0]:
#iterazione multipla dell'algoritmo HITS per osservare il valore di k in cui l'algoritmo inizia a convergere

delta_list = []

for k in range(1,31):
    
    sum_a = 0
    
    #calcolo score authority (influenzato dalla posizione della serp rispetto la keyword)
    for row in graph.itertuples(index=True, name='Pandas'):
        serp = row.SERP
        pos=row.POSITION
        key = row.KEYWORD
        if(k==1):
            authority[serp] = authority[serp] + pos
            sum_a = sum_a + authority[serp]
        else:
            authority[serp] = authority[serp] + hub[key]
            sum_a = sum_a + authority[serp]
    tot_a = 0

    for key in authority.keys():
        tot_a = tot_a + (authority[key]**2)

    tot_a = math.sqrt(tot_a)

    for key in authority.keys():
        authority[key] = round((authority[key]/tot_a),6)
        sum_a = sum_a + authority[key]

    sum_h = 0
    
    #calcolo score hub (influenzato dallo score dell'authority a cui esso punta)
    for row in graph.itertuples(index=True, name='Pandas'):
        serp = row.SERP
        key = row.KEYWORD
        hub[key] = hub[key] + authority[serp]
        sum_h = sum_h + hub[key]
        
    tot_h = 0
    for key in hub.keys():
        tot_h = tot_h + (hub[key]**2)

    tot_h = math.sqrt(tot_h)

    for key in hub.keys():
        hub[key] = round((hub[key]/tot_h),6)
        sum_h = sum_h + hub[key]
    
    avg_a = sum_a/len(authority)
    avg_h = sum_h/len(hub)
    
    d = round((avg_h - avg_a),3)
    delta_list.append(d)
    print("ITERAZIONE: ",k," DELTA: ",d, "SUM_A: ", sum_a , "SUM_H: ", sum_h )

In [0]:
plt.plot(delta_list)

In [0]:
#esecuzione algoritmo HITS con k=7. Ad ogni iterazione vengono aggiornati gli score per Authority e Hub

for k in range(1,8):
    
    #calcolo score authority (influenzato dalla posizione della serp rispetto la keyword)
    for row in graph.itertuples(index=True, name='Pandas'):
        serp = row.SERP
        pos=row.POSITION
        key = row.KEYWORD
        if(k==1):
            authority[serp] = authority[serp] + pos
        else:
            authority[serp] = authority[serp] + hub[key]
            
    # Normalizzazione dei vettori contenete i pesi per authority. 
    #Dividendo i vettori, componente per componente, per la radice della sommatoria dei quadrati di
    # tutti i loro elementi così che la loro somma dei quadrati sia unitaria
    
    tot_a = 0

    for key in authority.keys():
        tot_a = tot_a + (authority[key]**2)

    tot_a = math.sqrt(tot_a)

    for key in authority.keys():
        authority[key] = round((authority[key]/tot_a),8)

    #calcolo score hub (influenzato dallo score dell'authority a cui esso punta)
    for row in graph.itertuples(index=True, name='Pandas'):
        serp = row.SERP
        key = row.KEYWORD
        hub[key] = hub[key] + authority[serp]
    
    # Normalizzazione dei vettori contenete i pesi per hub.
    tot_h = 0
    for key in hub.keys():
        tot_h = tot_h + (hub[key]**2)

    tot_h = math.sqrt(tot_h)

    for key in hub.keys():
        hub[key] = round((hub[key]/tot_h),8)
    print("ITERAZIONE: ", k)

In [0]:
for elem in authority.keys():
    if(elem == 'lifegate.it'):
        print(authority[elem])

In [0]:
for elem in hub.keys():
    if(elem == 'antipasto giallo'):
        print(hub[elem])

In [0]:
authority_max = max(authority.keys(), key=(lambda k: authority[k]))
authority_min = min(authority.keys(), key=(lambda k: authority[k]))

hub_max = max(hub.keys(), key=(lambda k: hub[k]))
hub_min = min(hub.keys(), key=(lambda k: hub[k]))

print("MAX AUTHORITY: ",authority_max,"  HITS SCORE:",authority[authority_max])
print("MIN AUTHORITY: ",authority_min,"  HITS SCORE:",authority[authority_min])

print("MAX HUB: ",hub_max,"  HITS SCORE:",hub[hub_max])
print("MIN HUB: ",hub_min,"  HITS SCORE:",hub[hub_min])

In [0]:
df=[]
for elemen in authority.keys():
    b=authority[elemen]
    df.append(b)

In [0]:
myarray = np.asarray(df)

In [0]:
myarray= myarray.astype(float)

In [0]:
#il grafico mostra la distribuzione dei pesi delle authority

plt.plot(myarray)
plt.show()

In [0]:
h=sorted(myarray)
fit = stats.norm.pdf(h, np.mean(h), np.std(h))

plt.plot(h,fit,'-o')

plt.hist(h,normed=True)     

plt.show() 

In [0]:
#otteniamo il numero ottimale di bin applicando la regola di Sturge (1+(3.322*log(N)))

nbin=1+3.322*(math.log(len(myarray)))

In [0]:
nbin=int(nbin)

In [0]:
nbin

In [0]:
index = range(1,72813)
columns = ['auth']

dtype = [('a','float32')]
values = myarray
df = pd.DataFrame(values, index=index)

In [0]:
#viene aggiunta una colonna 'binned' per indicare il numero di bin a cui la serp appartiene

df['binned'] = pd.cut(df[0], nbin,labels=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38])

In [0]:
df['binned'].unique()

In [0]:
df['serps'] = set_s

In [0]:
df.loc[df['binned'] == 1].describe()

In [0]:
#Dizionario di dizionari
#KEY: bin index   VALUE: Dizionario SERP,KEYWORDS
index_serps = {}

#Dizionario contenete l'intersezione delle keyword collegate a serp appartenenti allo stesso cluster
#KEY: bin index   VALUE: Keywords intersection
index_key_set = {}

for i in range(2,39):
    start_time = time.time()
    if(not df.loc[df['binned'] == i].empty):
        df_t = df.loc[df['binned'] == i]
        key_serp = {}
        list_key = []
        for row in df_t.itertuples(index=True, name='Pandas'):
            serp = row.serps
            
            #considera solo le keyword che si trovano nelle prime 5 posizioni per poter caratterizzare la serp relativa come top site
            df_k = graph.loc[graph['SERP']== serp].loc[graph['POSITION'] >= 0.6] 
            
            #lista contenente i set delle keyword per ogni set, utilizzata successivamente per ottenere l'intersezione
            list_key.append(set(df_k['KEYWORD']))
            key_serp[serp] = set(df_k['KEYWORD'])
        
        #per ogni bin i contiene la coppia serp,keywords
        index_serps[i] = key_serp 
        
        #contiene l'intersezione delle keyword relative al bin i
        index_key_set[i] = set.intersection(*list_key) 
        
    print("Iterazione " , i ,"  --- %s seconds ---" % (time.time() - start_time))

In [0]:
str(index_serps.keys())

In [0]:
df.loc[df['binned'] == 38].describe()

In [0]:
for i in list(index_serps.keys()):
    print("BIN: ", i)
    print("SERPS: ", list(index_serps[i].keys()))
    print("N.KEYWORDS: ", len(index_key_set[i]))
    print("-------------------------------------------------------------------------")

In [0]:
list_index = [6,7,8,9,11,12,14,15,18,19,28,35,36,38]

In [0]:
for i in list_index:
    for j in list_index:
        if(j>i):
            print("KEY IN COMUNE TRA ",i,"(",len(index_key_set[i]),")"," E ", j,"(",len(index_key_set[j]),")")
            print(len(index_key_set[i].intersection(index_key_set[j])),"\n")


In [0]:
#crea un file che conterrà tutti i bin, composti da serp e le rispettive keyword
csv_tot = open("AllBin.csv","w", encoding = "utf-8")

#scrive in diversi file csv la lista di keyword raggruppate per ogni bin
for i in list(index_serps.keys()):
    if(len(index_key_set[i])!=0):
        est = ".csv"
        path =  "Clusters/ListKeywordsForBin_"+str(i)+est
        if not os.path.exists(os.path.dirname("./"+path)):
            os.makedirs(os.path.dirname("./"+path))
        csv = open(path, "w", encoding = "utf-8")  
        csv.write('')
        print("BIN: ", i)
        for elem in index_key_set[i]:
            csv.write(elem + "\n")
            for serp in index_serps[i].keys():
                csv_tot.write(serp+' ')
            csv_tot.write("\t" + elem + "\n")
        csv.close()
csv_tot.close()

In [0]:
str_111419 = ("misya.info buttalapasta.it youtube.com cookaround.com")

In [0]:
lst_key_index_111419 = []
set_111419 = index_key_set[11].intersection(index_key_set[14]).intersection(index_key_set[19])
len(set_111419)

j = 0
for elem in set_111419:
    if(j<=15):
        line = (str_111419.upper()+","+elem)
        lst_key_index_111419.append(line)
        j = j+1

In [0]:
#csv contenente l'intersezione tra le keyword appartenenti ai bin 11,14,19

csvFilename = "graphSample_111419.csv"
csv = open(csvFilename, "w", encoding = "utf-8") 
csv.write('')
for elem in lst_key_index_111419:
        csv.write(elem + "\n") 

In [0]:
#dizionario serps / keywords
#KEY: serps di ogni bin ottenuto  VALUE: keywords in comune tra le serp di ogni bin

serps_conctn = {}
for i in list_index:
    serps_conctn[i] = ""
    for elem in list(index_serps[i].keys()):
        if(serps_conctn[i] == ""):
            serps_conctn[i] = (elem+" ")
        else:
            serps_conctn[i] = (serps_conctn[i]+elem+" ")

In [0]:
serps_conctn

In [0]:
#crea una lista serps,keywords al fine di realizzare un sample del grafo per la visualizzazione

lst_key_index = []
for i in list_index:
    j = 0
    for elem in index_key_set[i]:
        if(j<=20):
            line = (serps_conctn[i]+","+elem)
            lst_key_index.append(line)
            j = j+1

In [0]:
csvFilename = "graphSample.csv"
csv = open(csvFilename, "w", encoding = "utf-8") 
csv.write('')
for elem in lst_key_index:
        csv.write(elem + "\n") 

In [0]:
g_volumes = pd.read_csv('gtrends_estimates.csv', sep='\t', header=None, encoding='utf-8', engine='python')

In [0]:
g_volumes = g_volumes.rename(columns={0:'KEYWORD',1:'VOLUME'})

In [0]:
gv_dict = {}

In [0]:
for row in g_volumes.itertuples(index=True, name='Pandas'):
    gv_dict[row.KEYWORD] = row.VOLUME

In [0]:
#Dizionario di dizionari
#KEY: bin index   VALUE: Dizionario KEYWORDS,VOLUME

bin_volume = {}
bin_list_volume = {}

In [0]:
for i in list_index:
    key_volume = {}
    for elem in index_key_set[i]:
        if(elem in gv_dict.keys()):
            key_volume[elem] = gv_dict[elem]
    key_volume_ord = sorted(key_volume.items(), key=lambda x: x[1], reverse=True)  #ordina il dizionario secondo i valori in modo decrescente
    bin_volume[i] = key_volume
    bin_list_volume[i] = key_volume_ord

In [0]:
import statistics
media=[]

for j in list_index:
    numbers = [bin_volume[j][key] for key in bin_volume[j]]
    mean_ = statistics.mean(numbers)
    media.append(mean_)
    print("BIN: ", j, " MEDIA VOLUMI: ",mean_)

In [0]:
plt.plot(range(0,len(media)),media,"-")
plt.show()

In [0]:
for i in list(bin_list_volume.keys()):
    est=".csv"
    path =  "topVolume/top100volume_"+str(i)+est
    if not os.path.exists(os.path.dirname("./"+path)):
        os.makedirs(os.path.dirname("./"+path))
    csv = open(path, "w", encoding = "utf-8")  
    csv.write('')
    k = 0
    for elem in bin_list_volume[i]:
        if(k<100):
            csv.write(str(elem) + "\n")
            k = k+1
    csv.close()